In [ ]:
import sys


no_of_cpu =8
max_cores =8
executor_mem = '56g'


Job_Name = 'Nishant: interaction_journal'


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType



In [ ]:
from pyspark.sql.functions import hash,col,regexp_replace,collect_list,concat_ws,initcap,count,least
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

In [ ]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpcustcoe"
pwd_prod="gpcustcoe@123"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

In [ ]:
def get_latest_progress(table_name,source):
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress",#table change
                 "server.port":"1150-1170"} 

#         list_tn=['id','header']
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime').filter(col('table_name').isin(table_name)).filter(col('status')=='Success')\
        .filter(col('records')!=0)
#         .filter(col('table_name')=='header')\
#             .filter(col('source')=='header')
        
        max_df=last_run.groupBy("table_name").agg(max("to_datetime").alias("datetime"))
        
        Max_last_run =max_df.select(min('datetime')).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))

    except Exception as E: 
        Max_last_run = datetime.datetime(1900, 10, 30, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

In [ ]:
# loading table as dataframe
def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
    "dbschema":"customermart",
    "dbtable": "header",
         "server.port":"1150-1170"}


header_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('golden_id','source_system_customer_id','active_flag','mapping_date').filter((col('active_flag')=='1')).filter((col('source_system_customer_id').isNotNull()))

In [ ]:
from pyspark.sql.window import Window

header_df = header_df.withColumn("rn", F.row_number()
        .over(Window.partitionBy('source_system_customer_id')
        .orderBy(F.col("mapping_date").desc(),F.col("golden_id").desc())))

header_df = header_df.filter(F.col("rn") == 1).drop("rn")

header_df=header_df.select('source_system_customer_id','golden_id','mapping_date')

In [ ]:
col_str=['golden_id','full_name','first_name','last_name','birth_date','gender','marital_status','weight_kg','tobacco_use_indicator','alcohol_use_indicator','height_cm']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark", "server.port":"1150-1170",
"password": "spark@456","dbschema":"customermart",
"dbtable": "customer_demographics"}
demog_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)

demog_df=demog_df.withColumn('customer_name',coalesce(col('full_name'),concat(col('first_name'),lit(' '),col('last_name'))))


In [ ]:
col_str=['golden_id','service_level']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark", "server.port":"1150-1170",
"password": "spark@456","dbschema":"customermart",
"dbtable": "customer_profile"}
profile_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)


In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123"
     ,"dbschema":"customermart"
    ,"dbtable": "address",
         "server.port":"1150-1170"}


address_retail_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('source_system_customer_id','address_type','city','last_updated')\
.filter((col('address_type')=='home'))

In [ ]:
from pyspark.sql.window import Window

address_retail_df = address_retail_df.withColumn("rn", F.row_number()
        .over(Window.partitionBy('source_system_customer_id','address_type')
        .orderBy(F.col("last_updated").desc())))

address_retail_df = address_retail_df.filter(F.col("rn") == 1).select('source_system_customer_id','city')

In [ ]:
report_date_str='2023-09-01'

In [ ]:
table_name='spav2_policy'
source='spav2_policy'

latest_progress=get_latest_progress(table_name,source)
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-2#currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

In [ ]:
col_str=["policy_number",'certificate_no', 'product_name', 'product_cd', 'reference_num',
         'pol_incept_date as policy_start_date', 'pol_exp_date as policy_end_date','source_system_name',
        'source_system_customer_id', 'lob', 'sum_insured', "uw_discount", 'premiumamount as premium',
        'veh_make', 'veh_model', 'veh_no as vehicle_registration_no', 'ncb_percent','producer_cd',
        'plan_type as plan_name','status','record_type_desc','commit_timestamp as binddate','veh_engine_no','veh_chassis_no',
         'times_renewed_cnt','min_pol_incept_date','no_of_lives']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123","dbschema":"customermart","server.port":"1150-1170",
"dbtable": "policy_retail"}
policy = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str).filter(col('binddate').between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
policy.columns

In [ ]:
col_str=['txt_parent_intermediary_cd as parent_producer_cd','txt_intermediary_cd as producer_cd']

gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123","dbschema":"public","server.port":"1150-1170",
"dbtable": "reference_gc_genmst_intermediary"}
producer_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
producer_df.columns

In [ ]:
# policy=policy.filter(F.date_add(col('policy_end_date'),90)>=to_timestamp(lit(report_date_str), format='yyyy-MM-dd'))

In [ ]:
lst_cancelled=['ACDC',
'ACDE',
'APC',
'APCP',
'APD',
'APRI',
'APU',
'DCC',
'ECDU',
'ECN',
'ECR',
'EPCE',
'ERFC',
'NCN',
'RCN']

In [ ]:
policy=policy.withColumn('record_type_desc', when((col('status').isin(lst_cancelled)),lit('CANCELLATION'))\
                           .otherwise(col('record_type_desc')))

In [ ]:
#Krishnan Badrinath
policy=policy.withColumn('assets',when(col('lob').isin('2W'),'2 Wheeler')\
                                        .when(col('lob').isin('Pvt Car'),'Car')\
                                        .when(col('lob').isin('Home'),'Home').otherwise(lit('-')))

In [ ]:
w = Window.partitionBy(['policy_number','certificate_no','policy_start_date', 'policy_end_date']).orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
policy_df = policy.withColumn('row',F.row_number().over(w))
policy_df = policy_df.filter(F.col('row')==1).drop(policy_df.row)


In [ ]:
policy_df=policy_df.withColumn('certificate_no',when(col('certificate_no').isin('0','0.0','/00000','000000','00000000'),lit('00000000')).otherwise(lit(col('certificate_no'))))\
.withColumn('policy_number',lpad(col('policy_number'),10,'0'))

In [ ]:
col_str=["risk1 as relation","reference_num",'insured_name as name','insured_dob as dob']
pdrdh_df=load_gpdb_jdbc(col_str,"policy_dh_risk_details_hlt",'public')

In [ ]:
col_str=['source','scoredate', 'score','income','dc_unified_id as golden_id']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe", "server.port":"1150-1170",
"password": "gpcustcoe@123","dbschema":"datamarts",
"dbtable": "credit_bureau_data"}
credit_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
credit_df.columns

In [ ]:
col_str=["policy_no", "renewal_prop_final_buckets","segment","month"]
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
"server.port": "1151-1170",
"dbschema":"customermart",
"dbtable": "propensity_score_jan_23_jan_24"}
ps = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
ps.columns

In [ ]:
ps=ps.withColumn('month',to_date(col('month'),'MMM-yy'))

from pyspark.sql.window import Window

ps = ps.withColumn("rn", F.row_number()
        .over(Window.partitionBy('policy_no')
        .orderBy(F.col("month").desc())))

ps = ps.filter(F.col("rn") == 1).drop("rn")

ps=ps.withColumn('renewal_propensity',concat(col('segment'),lit("("),col("renewal_prop_final_buckets"),lit(")")))

ps=ps.select('policy_no','renewal_propensity','month')

propensity_df=ps.withColumn('policy_no',lpad(trim(col('policy_no')),10,'0'))

In [ ]:
propensity_df=propensity_df.selectExpr('policy_no as policy_number', 'renewal_propensity')

In [ ]:
col_str=["dc_unified_id as golden_id","'Eligible for cross sell to Health' as is_eligible"]
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
 "server.port":"1150-1170",
"dbschema":"customermart",
"dbtable": "cross_sell_eligible_cust"}
cross_sell_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
cross_sell_df.columns

In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
    "dbschema":"customermart"
    ,"dbtable": "contactability",
         "server.port":"1150-1170"}

contactibilty_retail_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.selectExpr('source_system_customer_id','communication_sub_type','value','last_updated','source_system_name','verification_flag').filter((col('source_system_customer_id').isNotNull()))\
.filter(col('value').isNotNull()).filter(col('source_system_name')!='CREDIT_BUREAU')

In [ ]:
contactibilty_retail_df=contactibilty_retail_df.withColumn("rnk",when(col("source_system_name")=='GC',1)\
.otherwise(2))

In [ ]:
contactibilty_retail_df=contactibilty_retail_df.withColumn('mobile',when(col('communication_sub_type')=='mobile',col('value')).otherwise(None))
contactibilty_retail_df=contactibilty_retail_df.withColumn('email',when(col('communication_sub_type')=='email',col('value')).otherwise(None))

contactibilty_retail_df = contactibilty_retail_df.withColumn('email',when((lower(trim(col("email"))).rlike("^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(\.[a-zA-Z]{2,})?$") == False)\
                                                        & (lower(trim(col("communication_sub_type")))=='email'),lit(None)).otherwise(col('email')))



contactibilty_retail_df=contactibilty_retail_df.select('source_system_customer_id','mobile','email','rnk','last_updated','verification_flag')

In [ ]:
contactibilty_retail_df.columns

In [ ]:
contactibilty_retail_df=contactibilty_retail_df.join(header_df,'source_system_customer_id','inner')

In [ ]:
contactibilty_retail_df_email=contactibilty_retail_df.filter((col('email').isNotNull())).dropDuplicates(subset=["email","golden_id"])
contactibilty_retail_df_mob=contactibilty_retail_df.filter((col('mobile').isNotNull())).drop_duplicates(subset=["mobile","golden_id"])

In [ ]:
contactibilty_retail_df_email=contactibilty_retail_df_email.withColumn('email',lower(col('email')))

In [ ]:
# from pyspark.sql.window import Window

# contactibilty_retail_df_email = contactibilty_retail_df_email.withColumn("rn", F.row_number()
#         .over(Window.partitionBy('golden_id','email')
#         .orderBy(F.col("rnk").asc(),F.col("last_updated").desc(),F.col("email").desc_nulls_last())))

# contactibilty_retail_df_email = contactibilty_retail_df_email.filter(F.col("rn")<=2)

from pyspark.sql.window import Window

contactibilty_retail_df_email = contactibilty_retail_df_email.withColumn("rn", F.dense_rank()
        .over(Window.partitionBy('golden_id')
        .orderBy(F.col("rnk").asc(),F.col("last_updated").desc(),F.col("email").desc_nulls_last())))

contactibilty_retail_df_email = contactibilty_retail_df_email.filter(F.col("rn")<=2)

In [ ]:
# from pyspark.sql.window import Window

# contactibilty_retail_df_mob = contactibilty_retail_df_mob.withColumn("rn_mob", F.row_number()
#         .over(Window.partitionBy('golden_id','mobile')
#         .orderBy(F.col("rnk").asc(),F.col("last_updated").desc(),F.col("mobile").desc_nulls_last())))

# contactibilty_retail_df_mob = contactibilty_retail_df_mob.filter(F.col("rn_mob")<=2)

from pyspark.sql.window import Window

contactibilty_retail_df_mob = contactibilty_retail_df_mob.withColumn("rn_mob", F.dense_rank()
        .over(Window.partitionBy('golden_id')
        .orderBy(F.col("rnk").asc(),F.col("last_updated").desc(),F.col("mobile").desc_nulls_last())))

contactibilty_retail_df_mob = contactibilty_retail_df_mob.filter(F.col("rn_mob")<=2)

In [ ]:
contactibilty_retail_df_email=contactibilty_retail_df_email.select('golden_id','email','rn','verification_flag').filter((col('email').isNotNull())).drop_duplicates()
contactibilty_retail_df_mob=contactibilty_retail_df_mob.selectExpr('golden_id','mobile','rn_mob','verification_flag as verification_flag_mob').filter((col('mobile').isNotNull())).drop_duplicates()

In [ ]:
contactibilty_retail_df=contactibilty_retail_df_mob.join(contactibilty_retail_df_email,'golden_id','full_outer').drop_duplicates()

In [ ]:
contactibilty_retail_df.columns

In [ ]:
contactibilty_retail_df=contactibilty_retail_df.repartition('golden_id')

In [ ]:
col_str=['claim_no','claim_feature_concat','insured_name','claim_status','paid_amt_ind as claim_net',
         'paid_date_ind as claim_closed_date'
         ,'clm_create_date as claim_date','loss_reported_date as claim_reported_date',
'policy_no as policy_number','certificate_no',"pol_incept_date as policy_start_date",
         "pol_exp_date as policy_end_date",
         'claimant_name','loss_desc_txt as loss_description','customer_id as source_system_customer_id',
'coverage_desc as coverage_type','proposal_no as num_reference_no']

gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
"dbschema":"datamarts",
"dbtable": "producer_policy_product_wise_base_data",
"server.port": "1151-1171"}
crgp_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load().selectExpr(col_str)

#df_crp=df_crp.groupby('num_reference_no').agg(countDistinct(col('claim_no')).alias('claim_cnt'),sum(col('claim_gross').cast('double')).alias('claim_gross'))

crgp_df.columns

In [ ]:
crgp_df=crgp_df.withColumn('certificate_no',when(col('certificate_no').isin('0','0.0','/00000','000000','00000000'),lit('00000000')).otherwise(lit(col('certificate_no'))))\
.withColumn('policy_number',lpad(col('policy_number'),10,'0'))

crgp_df=crgp_df.withColumn('policy_start_date',to_date(col('policy_start_date'),'dd-MM-yyyy'))\
.withColumn('policy_end_date',to_date(col('policy_end_date'),'dd-MM-yyyy'))

In [ ]:
crgp_df=crgp_df.withColumn('policy_start_date',to_date(col('policy_start_date'),'dd-MM-yyyy'))\
.withColumn('policy_end_date',to_date(col('policy_end_date'),'dd-MM-yyyy'))\
.withColumn('claim_date',to_date(col('claim_date'),'dd-MM-yyyy'))


In [ ]:
col_str=["nameofpreexistingdisease as ped","num_reference_number as reference_num"]
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
"server.port": "1151-1170",
"dbschema":"public",
"dbtable": "underwriting_dh_cnfgtr_otherdt_grid_tab_hlt"}
pd_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
pd_df.columns

In [ ]:
col_str=['feedback_title','feedback_date as date','score','golden_id']
# "current_year"
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
"server.port": "1151-1170",
"dbschema":"customermart",
"dbtable": "feedback_spa_v2"}
feedback_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
feedback_df.columns

## Active

In [ ]:
pdf=policy_df.select('source_system_customer_id','policy_end_date','record_type_desc','policy_number','certificate_no')

is_policy=header_df.join(pdf,'source_system_customer_id','inner')

In [ ]:
header_df=is_policy.select('golden_id','source_system_customer_id','mapping_date')

In [ ]:
is_policy.columns

In [ ]:
is_active_df=is_policy.withColumn('cust_status', when((col('policy_end_date') >=current_date()) &(col('record_type_desc')!='CANCELLATION') , lit(1)).otherwise(lit(0)))\

In [ ]:
is_cust_active=is_active_df.groupBy('golden_id').agg(max('cust_status').alias("is_active"))


In [ ]:
is_cust_active=is_cust_active.withColumn('is_active',when(is_cust_active['is_active']==1,True).otherwise(False))

# policyDetails

In [ ]:
# policy_details=header_df.join(policy_df,'source_system_customer_id',"inner").drop_duplicates()

In [ ]:
policy_details=policy_df.join(header_df,'source_system_customer_id',"left").drop_duplicates()

In [ ]:
policy_details=policy_details.join(producer_df,'producer_cd',"left").drop_duplicates()

In [ ]:
# from pyspark.sql.window import Window

# w = Window.partitionBy(['golden_id','policy_number','certificate_no','policy_start_date', 'policy_end_date']).orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
# policy_details = policy_details.withColumn('row',F.row_number().over(w))
# policy_details = policy_details.filter(F.col('row')==1).drop(policy_details.row).drop_duplicates()

from pyspark.sql.window import Window

w = Window.partitionBy(['policy_number','certificate_no','policy_start_date', 'policy_end_date']).orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
policy_details = policy_details.withColumn('row',F.row_number().over(w))
policy_details = policy_details.filter(F.col('row')==1).drop(policy_details.row).drop_duplicates()

In [ ]:
crgp_df=crgp_df.drop_duplicates()

In [ ]:
from pyspark.sql.functions import sum,avg,max
group_cols = ["policy_number", "certificate_no",'policy_start_date', 'policy_end_date']
df_p=crgp_df.groupBy(group_cols).agg(countDistinct("claim_no").alias("claim_count"),sum("claim_net").alias("total_claim_amount"))

In [ ]:
policy_details=policy_details.join(df_p,group_cols,'left')

In [ ]:
group_cols = ["policy_number", "certificate_no",'policy_start_date', 'policy_end_date']

In [ ]:
policy_details=policy_details.join(crgp_df,group_cols,"left_outer").drop(crgp_df.num_reference_no)\
.drop(crgp_df.policy_number).drop(crgp_df.certificate_no).drop(crgp_df.source_system_customer_id).drop(crgp_df.policy_start_date)\
.drop(crgp_df.policy_end_date)

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "policy_dh_risk_headers_mot",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

mot_uw_mot = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("reference_num",'variant as vehicle_variant', "nominee_name as nominee_name_mot",'nomineeage as nomineeage_mot', 'relationship as relationship_mot', 'relationship_to_the_insured as relationship_to_the_insured_mot')\
                   .dropDuplicates()

mot_uw_mot.columns

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "underwriting_dh_cnfgtr_otherdt_grid_tab_hlt",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

uw_hlt = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("num_reference_number as reference_num", "nomineename as nomineename_hlt",'age as nom_age_hlt',
                               'dateofbirth as nom_dob_hlt', "relationshipwithinsured as nomi_rel_hlt",'appointeename','relationshipappnom','memberid')\
                   .filter(col('nomineename_hlt').isNotNull())

uw_hlt.columns


from pyspark.sql.window import Window

w = Window.partitionBy(['reference_num']).orderBy([F.col('memberid').asc()])
uw_hlt = uw_hlt.withColumn('row',F.row_number().over(w))
uw_hlt = uw_hlt.filter(F.col('row')==1).drop(uw_hlt.row)


uw_hlt=uw_hlt.withColumn('nomineename_hlt', trim(uw_hlt.nomineename_hlt))
uw_hlt.columns

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "policy_dh_risk_details_lta",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

lta = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("reference_num", 'nominee_name as nominee_name_lta', 'relationship as relationship_lta', 'dob as dob_lta','risk_detail_serial').filter(col('risk_detail_serial')==0)\
                   .dropDuplicates()

lta.columns

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "underwriting_dh_cnfgtr_otherdt_grid_tab_trv",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

uw_trv = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("num_reference_number as reference_num", "nomineedob as nomineedob_trv", 'nomineename as nomineename_trv', 'nomineerelationship as nomineerelationship_trv', 'nomineegender as nomineegender_trv','seq_no')

from pyspark.sql.window import Window

w = Window.partitionBy(['reference_num']).orderBy([F.col('seq_no').asc()])
uw_trv = uw_trv.withColumn('row',F.row_number().over(w))
uw_trv = uw_trv.filter(F.col('row')==1).drop(uw_trv.row)

uw_trv.columns

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "underwriting_dh_cnfgtr_otherdt_grid_tab_mot",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

appointee_mot = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("num_reference_number as reference_num","nameoftheappointee","relationshipappnom as relationshipappnom_mot")

appointee_mot.columns

In [ ]:
policy_details =policy_details.join(mot_uw_mot, "reference_num","left")
policy_details =policy_details.join(uw_hlt, "reference_num","left")
policy_details =policy_details.join(uw_trv, "reference_num","left")
policy_details=policy_details.join(lta, "reference_num","left")
policy_details=policy_details.join(appointee_mot,"reference_num","left")
policy_details.columns

In [ ]:
policy_details=policy_details.withColumn('nominee_name', coalesce(col('nomineename_hlt'), col('nominee_name_mot'), col('nomineename_trv'), col('nominee_name_lta')))\
.withColumn('nominee_dob', coalesce(col('nomineedob_trv'), col('nom_dob_hlt')))\
.withColumn('nominee_age', coalesce(col('nomineeage_mot'), col('nom_age_hlt')))\
.withColumn('nominee_gender', coalesce(col('nomineegender_trv')))\
.withColumn('nominee_relation', coalesce(col('nomi_rel_hlt'), col('relationship_mot'),col('relationship_to_the_insured_mot'),col('relationship_lta'), col('nomineerelationship_trv')))\
.withColumn('appointee_name', coalesce(col('appointeename'), col('nameoftheappointee')))\
.withColumn('apppointee_relation', coalesce(col('relationshipappnom_mot'), col('relationshipappnom')))


In [ ]:
policy_details=policy_details.withColumn('appointee_name',when(col('appointee_name').isNull(),lit('-')).otherwise(col('appointee_name')))\
.withColumn('apppointee_relation',when(col('apppointee_relation').isNull(),lit('-')).otherwise(col('apppointee_relation')))\
.withColumn('nominee_name',when(col('nominee_name').isNull(),lit('-')).otherwise(col('nominee_name')))\
.withColumn('nominee_gender',when(col('nominee_gender').isNull(),lit('-')).otherwise(col('nominee_gender')))\
.withColumn('nominee_relation',when(col('nominee_relation').isNull(),lit('-')).otherwise(col('nominee_relation')))\
.withColumn('nominee_age',when(col('nominee_age').isNull(),lit('-')).otherwise(col('nominee_age')))

In [ ]:
policy_details_main=policy_details\
.withColumn('claim_count',when(col('claim_count').isNull(),lit('0')).otherwise(col('claim_count')))\
.withColumn('total_claim_amount',when(col('total_claim_amount').isNull(),lit('0.0')).otherwise(col('total_claim_amount')))\
.withColumn('uw_discount',when(col('uw_discount').isNull(),lit('0.0')).otherwise(col('uw_discount')))\
.withColumn('claim_no',when(col('claim_no').isNull(),lit('-')).otherwise(col('claim_no')))\
.withColumn('loss_description',when(col('loss_description').isNull(),lit('-')).otherwise(col('loss_description')))\
.withColumn('coverage_type',when(col('coverage_type').isNull(),lit('-')).otherwise(col('coverage_type')))\
.withColumn('claim_net',when(col('claim_net').isNull(),lit(None)).otherwise(col('claim_net')))\
.withColumn('total_claim_amount',when(col('total_claim_amount').isNull(),lit('0.0')).otherwise(col('total_claim_amount')))\
.withColumn('claim_closed_date',when(col('claim_closed_date').isNull(),lit(None)).otherwise(col('claim_closed_date')))\


In [ ]:
policy_details_main=policy_details_main.withColumn('nominee_dob',to_date(col('nominee_dob'),'dd-MM-yyyy'))

In [ ]:
policy_details_main=policy_details_main.drop_duplicates()

In [ ]:
# policy_details_main=policy_details_main.withColumn('current_year', when(year(col('policy_start_date'))==year(current_date()), lit(True)).otherwise(lit(False)))\
# .withColumn('status', when(col('policy_end_date') >=current_date(), lit('Active'))\
#             .when(months_between(current_date(),col('policy_end_date'))<=3,lit('DueForRenewal')).otherwise(lit('Inactive')))\
# .withColumn('status_enum', when(col('policy_end_date') >=current_date(), lit(1))\
#             .when(col("status")=='DueForRenewal',lit(2)).otherwise(lit(0)))

In [ ]:
policy_details_main=policy_details_main.withColumn('current_year', when(year(col('policy_start_date'))==year(current_date()), lit(True)).otherwise(lit(False)))\
.withColumn('status', when((months_between(col('policy_end_date'),current_date())<=3)&(months_between(col('policy_end_date'),current_date())>0),lit('DueForRenewal'))\
            .when(col('policy_end_date') >=current_date(), lit('Active'))\
            .otherwise(lit('Inactive')))\
.withColumn('status_enum',when(col("status")=='DueForRenewal',lit(2))\
            .when(col('policy_end_date') >=current_date(), lit(1))\
            .otherwise(lit(0)))

In [ ]:
policy_details_main=policy_details_main.withColumn('status', when((col('record_type_desc').isin('CANCELLATION')),lit('Inactive'))\
                           .otherwise(col('status')))\
.withColumn('status_enum', when((col('record_type_desc').isin('CANCELLATION')),lit(0))\
                           .otherwise(col('status_enum')))

In [ ]:
policy_details_main=policy_details_main.withColumn('primary_key', concat_ws('',trim(col('policy_number')), trim(col('certificate_no')),trim(col('claim_feature_concat')),trim(col('policy_start_date')), trim(col('policy_end_date'))))

In [ ]:
from pyspark.sql.window import Window

w = Window.partitionBy('primary_key').orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
policy_details_main = policy_details_main.withColumn('row',F.row_number().over(w))
policy_details_main = policy_details_main.filter(F.col('row')==1).drop(policy_details_main.row).drop_duplicates()

In [ ]:
policy_details_main.persist()
records=policy_details_main.count()

In [ ]:
output_index = 'spa_get_policy_details_v2'#cpa_all_prod_fy20,cpa_all_prod_fy21
schema = 'customermart'

import sys
try:
    
     policy_details_main.write.option("truncate", "true").format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 'success'
    print(x)

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": user_prod,
                    "password": pwd_prod,
                    "dbschema": "customermart",
                    "dbtable": "spa_get_policy_details_v2",#policy_retail_staging_2018_03,
                    "server.port": '1150-1170'}
policy_details_main = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()


# .filter(~(col('product_name').isin(l)))

In [ ]:
policy_details_main=policy_details_main.select('policy_number', 'certificate_no', 'source_system_customer_id', 'golden_id', 'product_name', 'product_cd', 'reference_num', 'policy_start_date', 'policy_end_date', 'lob', 'sum_insured', 'uw_discount', 'premium', 'veh_make', 'veh_model', 
                                     'vehicle_registration_no', 'ncb_percent', 'plan_name', 'status', 'record_type_desc', 'binddate','veh_engine_no', 'veh_chassis_no', 'claimant_name', 'claim_reported_date', 'claim_no',
                                     'claim_closed_date', 'loss_description', 'coverage_type', 'claim_net', col('vehicle_variant').cast('String'), col('nominee_name').cast('String'), col('nominee_relation').cast('String'), col('nominee_dob').cast('String'), col('nominee_age').cast('String'), col('appointee_name').cast('String'), col('apppointee_relation').cast('String'), 'current_year', 'status_enum', 'claim_count', 'total_claim_amount','producer_cd','parent_producer_cd','primary_key')

In [ ]:
policy_details_main_f=policy_details_main.selectExpr('golden_id',"current_year","policy_number as policy_no","certificate_no","lob","status","status_enum","product_name","product_cd as product_code","reference_num as proposal_no",
"source_system_customer_id as customer_id","policy_start_date","policy_end_date","sum_insured","uw_discount as discount","premium","claim_count","total_claim_amount","veh_make as vehicle_make","veh_model as vehicle_model",
"vehicle_variant","vehicle_registration_no","plan_name","ncb_percent as ncb","nominee_name","nominee_relation","nominee_dob","nominee_age","appointee_name",'binddate',
"apppointee_relation","claim_no","claim_reported_date","claim_closed_date","claimant_name","loss_description","coverage_type","claim_net as claim_amount_paid",'producer_cd','parent_producer_cd','primary_key')

In [ ]:
policy_details_main_f1=policy_details_main_f.select('golden_id', 'current_year', 'policy_no', 'certificate_no', 'lob', 'status', 'status_enum', 'product_name', 'product_code', 'proposal_no', 'customer_id', 'policy_start_date', 'policy_end_date', col('sum_insured').cast('long'), 'discount', 'premium', 'claim_count', 'total_claim_amount', 'vehicle_make', 'vehicle_model', 'vehicle_variant', 'vehicle_registration_no', 'plan_name', col('ncb').cast('String'), 'nominee_name', 'nominee_relation', 'nominee_dob', 'nominee_age', 'appointee_name', 'apppointee_relation', 'claim_no', 'claim_reported_date', 'claim_closed_date', 'claimant_name', 'loss_description', 'coverage_type', 'claim_amount_paid','binddate','primary_key')

In [ ]:
policy_details_main_f1=policy_details_main_f1.withColumn('policy_start_date',date_format(to_utc_timestamp(col('policy_start_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('policy_end_date',date_format(to_utc_timestamp(col('policy_end_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('claim_reported_date',date_format(to_utc_timestamp(col('claim_reported_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('claim_closed_date',date_format(to_utc_timestamp(col('claim_closed_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('nominee_dob',date_format(to_utc_timestamp(col('nominee_dob'),"UTC"),'yyyy-MM-dd'))

In [ ]:
policy_details_main_f1.columns

In [ ]:
policy_details_main_f1=policy_details_main_f1.drop_duplicates()

In [ ]:
# policy_details_main_f_active=policy_details_main_f1.filter(F.date_add(col('policy_end_date'),90)>=to_timestamp(lit(report_date_str), format='yyyy-MM-dd'))
# policy_details_main_f_inactive=policy_details_main_f1.filter(F.date_add(col('policy_end_date'),90)<to_timestamp(lit(report_date_str), format='yyyy-MM-dd'))

In [ ]:
target_index='get_spa_policy_details_active'
print(target_index)

In [ ]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'get_spa_policy_details_active'
#########delete index if already exists########################################################################################################################################################################

In [ ]:
target_index_doc_type = target_index +'/'+ target_doc_type

In [ ]:
target_index_doc_type

In [ ]:
######### create mapping##################################################################################################################################################################################################################################################################
mapping_output = {target_doc_type:{"properties": {
"golden_id": {
"type": "keyword"
},
"current_year": {
"type": "keyword","index":"false"
},
"policy_no": {
"type": "keyword",
},
"certificate_no": {
"type": "keyword",
},
"lob": {
"type": "keyword","index":"false"
},
"status": {
"type": "keyword","index":"false"
},
"status_enum": {
"type": "keyword"
},
"product_name": {
"type": "keyword","index":"false"
},
"product_code": {
"type": "keyword","index":"false"
},
"proposal_no": {
"type": "keyword","index":"false"
},
"customer_id": {
"type": "keyword","index":"false"
},
"policy_start_date": {
"type":"keyword","index":"false"
},
"policy_end_date": {
"type": "keyword","index":"false"
},
"sum_insured": {
"type": "keyword","index":"false"
},
"discount": {
"type": "keyword","index":"false"
},
"premium": {
"type": "keyword","index":"false"
},
"claim_count": {
"type":"keyword","index":"false"
},
"total_claim_amount": {
"type": "keyword","index":"false"
},
"vehicle_make": {
"type": "keyword","index":"false"
},
"vehicle_model": {
"type": "keyword","index":"false"
},
"vehicle_variant": {
"type":"keyword","index":"false"
},
"vehicle_registration_no": {
"type": "keyword","index":"false"
},
"plan_name": {
"type": "keyword","index":"false"
},
"ncb": {
"type": "keyword","index":"false"
},
"nominee_name": {
"type": "keyword","index":"false"
},
"nominee_relation": {
"type":"keyword","index":"false"
},
"nominee_dob": {
"type": "keyword","index":"false"
},
"nominee_age": {
"type": "keyword","index":"false"
},
"appointee_name": {
"type": "keyword","index":"false"
},
"apppointee_relation": {
"type":"keyword","index":"false"
},
"claim_no": {
"type": "keyword","index":"false"
},
"claim_reported_date": {
"type": "keyword","index":"false"
},
"claim_closed_date": {
"type": "keyword","index":"false"
},
"claimant_name": {
"type": "keyword","index":"false"
},
"loss_description": {
"type":"keyword","index":"false"
},
"coverage_type": {
"type": "keyword","index":"false"
},
"claim_amount_paid": {
"type": "keyword","index":"false"
},
"binddate": {
"type": "keyword","index":"false"
},
"primary_key": {
"type": "keyword"
}}
}}


URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index+'/_mapping/'+target_doc_type
headers = {'Content-Type': 'application/json'}
r = requests.put(URL, data=json.dumps(mapping_output), headers=headers)
print(r)

In [ ]:
try:
     policy_details_main_f1.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
        .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
        .option("es.write.operation","upsert").option("es.mapping.id",'primary_key').mode("append").save() 
except Exception as e :
    x = e
else: 
    x = 200 #success
print(x)

# policyDetailsSummary

In [ ]:
policy_details_main_f=policy_details_main_f.filter(col('golden_id').isNotNull())

In [ ]:
policy_summary=policy_details_main.selectExpr('golden_id',"policy_number as policy_no","producer_cd as producer_code",'parent_producer_cd as parent_producer_code',"certificate_no","lob","status","status_enum","product_name","product_cd as product_code","policy_start_date",
"policy_end_date",'sum_insured',"uw_discount as discount","premium","claim_count","total_claim_amount",'binddate','reference_num')

In [ ]:
policy_summary=policy_summary.withColumn('discount',when(col('discount').isNull(),lit('0.0')).otherwise(col('discount')))\
.withColumn('claim_count',when(col('claim_count').isNull(),lit('0')).otherwise(col('claim_count')))\
.withColumn('total_claim_amount',when(col('total_claim_amount').isNull(),lit('0.0')).otherwise(col('total_claim_amount')))

In [ ]:
policy_summary=policy_summary.withColumn('policy_start_date',date_format(to_utc_timestamp(col('policy_start_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('policy_end_date',date_format(to_utc_timestamp(col('policy_end_date'),"UTC"),'yyyy-MM-dd'))

In [ ]:
policy_summary=policy_summary.filter(col('golden_id').isNotNull())

In [ ]:
policy_summary=policy_summary.drop_duplicates()

In [ ]:
from pyspark.sql.window import Window

w = Window.partitionBy(['golden_id','policy_no','certificate_no']).orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
policy_summary = policy_summary.withColumn('row',F.row_number().over(w))
policy_summary = policy_summary.filter(F.col('row')==1).drop(policy_summary.row).drop_duplicates()

In [ ]:
policy_summary=policy_summary.withColumn('primary_key', concat_ws('',trim(col('golden_id')),trim(col('policy_no')), trim(col('certificate_no'))))

In [ ]:
from pyspark.sql.window import Window

w = Window.partitionBy('primary_key').orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
policy_summary = policy_summary.withColumn('row',F.row_number().over(w))
policy_summary = policy_summary.filter(F.col('row')==1).drop(policy_summary.row).drop_duplicates()

In [ ]:
policy_summary=policy_summary.selectExpr('golden_id',"policy_no","producer_code",'parent_producer_code',"certificate_no","lob","status","status_enum","product_name","product_code","policy_start_date",
"policy_end_date","sum_insured","discount","premium","claim_count","total_claim_amount",'binddate','primary_key')

In [ ]:
policy_summary.persist()
policy_summary.count()

In [ ]:
max_commit_timestamp =policy_summary.select(max('binddate')).first()[0]

In [ ]:
max_commit_timestamp.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
# policy_summary_active=policy_summary.filter(F.date_add(col('policy_end_date'),90)>=to_timestamp(lit(report_date_str), format='yyyy-MM-dd'))
# policy_summary_inactive=policy_summary.filter(F.date_add(col('policy_end_date'),90)<to_timestamp(lit(report_date_str), format='yyyy-MM-dd'))

In [ ]:
target_index='get_spa_policy_details_summary_active'
print(target_index)

In [ ]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'get_spa_policy_details_summary_active'
#########delete index if already exists########################################################################################################################################################################

In [ ]:
target_index_doc_type = target_index +'/'+ target_doc_type

In [ ]:
target_index_doc_type

In [ ]:
######### create mapping##################################################################################################################################################################################################################################################################
mapping_output = {target_doc_type:{"properties": {
"golden_id": {
"type": "keyword"
},
"policy_no": {
"type": "keyword","index":"false"
},
"producer_code": {
"type": "keyword","index":"false"
},
"parent_producer_code": {
"type": "keyword","index":"false"
},
"certificate_no": {
"type":"keyword","index":"false"
},
"lob": {
"type": "keyword","index":"false"
},
"status": {
"type": "keyword","index":"false"
},
"status_enum": {
"type": "keyword"
},
"product_name": {
"type": "keyword","index":"false"
},
"product_code": {
"type": "keyword","index":"false"
},
"policy_start_date": {
"type":"keyword","index":"false"
},
"policy_end_date": {
"type": "keyword","index":"false"
},
"sum_insured": {
"type": "keyword","index":"false"
},
"discount": {
"type": "keyword","index":"false"
},
"premium": {
"type": "keyword","index":"false"
},
"claim_count": {
"type":"keyword","index":"false"
},
"total_claim_amount": {
"type": "keyword","index":"false"
},
"binddate": {
"type": "keyword","index":"false"
},
"primary_key": {
"type": "keyword"
}}
}}



URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index+'/_mapping/'+target_doc_type
headers = {'Content-Type': 'application/json'}
r = requests.put(URL, data=json.dumps(mapping_output), headers=headers)
print(r)

In [ ]:
try:
     policy_summary.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
        .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
        .option("es.write.operation","upsert").option("es.mapping.id",'primary_key').mode("append").save() 
except Exception as e :
    x = e
else: 
    x = 200 # success
print(x)

# GET POLICY LIST

In [ ]:
policy_details_main=policy_details_main.filter(col('golden_id').isNotNull())

In [ ]:
col_str=['NUM_REFERENCE_NO as reference_num','WATTS_POLICY_NO as IPDS_POLICY_NUMBER']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"server.port": "1151-1170",
"dbschema":"registers",
"dbtable": "premium_register"}
pr_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)

In [ ]:
policy_retail_base=policy_details_main.join(pr_df,'reference_num','inner')

In [ ]:
policy_retail_base=policy_retail_base.join(contactibilty_retail_df,'golden_id','left')

In [ ]:
policy_retail_base_main=policy_retail_base.selectExpr('mobile as MOBILE_NUMBER','email as EMAIL_ID','IPDS_POLICY_NUMBER',
                                                 'policy_number as POLICY_NUMBER','claim_no as CLAIM_NUMBER',
                                                 'certificate_no as CERTIFICATE_NUMBER',
'reference_num as PROPOSAL_NUMBER',"'-' as PROPOSAL_STATUS","'-' as PROPOSAL_DESCRIPTION","'-' as CUSTOMER_NAME",
                                                 'source_system_customer_id as CUSTOMER_ID','binddate'
,'vehicle_registration_no as VEHICLE_REGISTRATION_NUMBER','sum_insured as SUM_INSURED','lob as LOB',
                                                 'product_cd as PRODUCT_CODE','product_name as PRODUCT_NAME',
                                                 'policy_start_date as POLICY_START_DATE','policy_end_date as POLICY_END_DATE','golden_id as DC_UNIFIED_ID')


In [ ]:
policy_retail_base_main=policy_retail_base_main.drop_duplicates()

In [ ]:
policy_retail_base_main.persist()
policy_retail_base_main.count()

In [ ]:
output_index = 'get_policy_list_spa_2'#cpa_all_prod_fy20,cpa_all_prod_fy21
schema = 'customermart'

import sys
try:
    
     policy_retail_base_main.write.option("truncate", "true").format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 'success'
    print(x)

In [ ]:
# gscPythonOptions = {
#     "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
#     "dbschema":"customermart","dbtable": "get_policy_list_spa_2","server.port":"1150-1170"}

# policy_retail_base_main = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()

In [ ]:
from pyspark.sql.window import Window

policy_retail_base_main=policy_retail_base_main.withColumn("CLAIM_NUMBER", collect_set(col("CLAIM_NUMBER")).over(Window.partitionBy("PROPOSAL_NUMBER")))


In [ ]:
policy_retail_base_main=policy_retail_base_main.withColumn("LOAD_DATE",current_date())

In [ ]:
policy_retail_base_main=policy_retail_base_main.selectExpr('MOBILE_NUMBER', 'EMAIL_ID', '0 as IPDS_POLICY_NUMBER', 'POLICY_NUMBER', 'CLAIM_NUMBER', 'CERTIFICATE_NUMBER', 'PROPOSAL_NUMBER', 'PROPOSAL_STATUS', 'PROPOSAL_DESCRIPTION', 'CUSTOMER_NAME', 'CUSTOMER_ID', 'VEHICLE_REGISTRATION_NUMBER', 'SUM_INSURED', 'LOB', 'PRODUCT_CODE', 'PRODUCT_NAME', 'POLICY_START_DATE', 'POLICY_END_DATE', 'DC_UNIFIED_ID',
                                              '0.0 as PREMIUM_AMOUNT_INR_WITHOUT_TAX','0.0 as CGST_AMOUNT','0.0 as IGST_AMOUNT','binddate',
'0.0 as SGST_AMOUNT','0.0 as PREMIUM_WITH_TAX','null as BRANCH','null as RECORD_TYPE_DESC','null as RENL_CERT_NO','null as EFF_DT_SEQ_NO',
'null as TXT_TIMES_RENEWED', 'null as PREVIOUS_POL_START_DATE', 'null as PREVIOUS_POL_EXPIRY_DATE','null as PRODUCER_CD','null as PRODUCER_NAME','null as POLICY_STATUS',
                                                   'null as DRIVING_LICENSE_NUMBER', 'null as PORTABILITY_FLAG','LOAD_DATE')

In [ ]:
policy_retail_base_main1=policy_retail_base_main.select(col('DC_UNIFIED_ID').cast('long'), 'CUSTOMER_ID', 'PROPOSAL_STATUS', 
                                                   col('PROPOSAL_NUMBER').cast('string'), 'PRODUCT_CODE','POLICY_NUMBER',
                                                   'CERTIFICATE_NUMBER', col('POLICY_START_DATE').cast('string'), 
                                                   col('POLICY_END_DATE').cast('string'), 
                                                   'PRODUCT_NAME', col('PREMIUM_AMOUNT_INR_WITHOUT_TAX').cast('double'),
                                                   col('CGST_AMOUNT').cast('double'), 'binddate',
                                                   col('IGST_AMOUNT').cast('double'), col('SGST_AMOUNT').cast('double'), 
col('PREMIUM_WITH_TAX').cast('double'), col('SUM_INSURED').cast('double'),col('PRODUCER_NAME').cast('string'),
'CLAIM_NUMBER', col('PRODUCER_CD').cast('string'), col('BRANCH').cast('string'), col('RECORD_TYPE_DESC').cast('string')
                                                   , 'LOB', 'VEHICLE_REGISTRATION_NUMBER', 'IPDS_POLICY_NUMBER', 
                                                   col('POLICY_STATUS').cast('string'), col('PROPOSAL_DESCRIPTION').cast('string'), col('RENL_CERT_NO').cast('string'), col('EFF_DT_SEQ_NO').cast('string'), 
                                                   col('TXT_TIMES_RENEWED').cast('string'), col('PREVIOUS_POL_START_DATE').cast('string'), col('PREVIOUS_POL_EXPIRY_DATE').cast('string'),
                                                   col('DRIVING_LICENSE_NUMBER').cast('string'), col('PORTABILITY_FLAG').cast('string'), col('MOBILE_NUMBER').cast('string'), col('EMAIL_ID').cast('string'), 
                                                   col('CUSTOMER_NAME').cast('string'), col('LOAD_DATE').cast('string'))

In [ ]:
from pyspark.sql.window import Window

w = Window.partitionBy(['MOBILE_NUMBER', 'EMAIL_ID','IPDS_POLICY_NUMBER','POLICY_NUMBER', 'CERTIFICATE_NUMBER','CUSTOMER_ID','DC_UNIFIED_ID'])\
.orderBy([F.col('binddate').desc(),F.col('PROPOSAL_NUMBER').desc()])
policy_retail_base_main = policy_retail_base_main.withColumn('row',F.row_number().over(w))
policy_retail_base_main = policy_retail_base_main.filter(F.col('row')==1).drop(policy_retail_base_main.row).drop_duplicates()

In [ ]:
policy_retail_base_main1=policy_retail_base_main1.withColumn('primary_key', concat_ws('',trim(col('MOBILE_NUMBER')),trim(col('EMAIL_ID')), trim(col('IPDS_POLICY_NUMBER')),trim(col('POLICY_NUMBER')),trim(col('CERTIFICATE_NUMBER')), trim(col('CUSTOMER_ID')), trim(col('DC_UNIFIED_ID'))))

In [ ]:
cert=['/00000','0.0','000000','00000000',None]

policy_retail_base_main1=policy_retail_base_main1.withColumn('IPDS_POLICY_NUMBER',when(col('IPDS_POLICY_NUMBER').isin(0),lit(None))\
                                            .when(col('IPDS_POLICY_NUMBER').isNull(),lit(None))\
                                            .otherwise(col('IPDS_POLICY_NUMBER')))\
.withColumn('CERTIFICATE_NUMBER',when(col('CERTIFICATE_NUMBER').isin(cert),lit("-"))\
                                            .when(col('CERTIFICATE_NUMBER').isNull(),lit("-"))\
                                            .otherwise(col('CERTIFICATE_NUMBER')))

In [ ]:
policy_retail_base_main1=policy_retail_base_main1.withColumn('EMAIL_ID',lower(col('EMAIL_ID')))

In [ ]:
policy_retail_base_main1=policy_retail_base_main1.repartition('primary_key')

In [ ]:
# policy_retail_base_main_active=policy_retail_base_main1.filter(F.date_add(col('POLICY_END_DATE'),90)>=to_timestamp(lit(report_date_str), format='yyyy-MM-dd'))
# policy_retail_base_main_inactive=policy_retail_base_main1.filter(F.date_add(col('POLICY_END_DATE'),90)<to_timestamp(lit(report_date_str), format='yyyy-MM-dd'))

In [ ]:
target_index='spa_get_policy_list_active'
print(target_index)

In [ ]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'spa_get_policy_list_active'
#########delete index if already exists########################################################################################################################################################################

In [ ]:
target_index_doc_type = target_index +'/'+ target_doc_type

In [ ]:
target_index_doc_type

In [ ]:
policy_retail_base_main1=policy_retail_base_main1.drop_duplicates()

In [ ]:
######### create mapping##################################################################################################################################################################################################################################################################
mapping_output = {target_doc_type:{"properties": {
"CUSTOMER_ID": {
  "type": "keyword",
},
"PROPOSAL_STATUS": {
  "type": "keyword","index":"false"
},
"PROPOSAL_NUMBER": {
  "type": "keyword",
},
"PRODUCT_CODE": {
  "type": "keyword","index":"false"
},
"POLICY_NUMBER": {
  "type": "keyword",
},
"CERTIFICATE_NUMBER": {
  "type": "keyword",
},
"POLICY_START_DATE": {
  "type": "keyword","index":"false"
},
"POLICY_END_DATE": {
 "type": "keyword","index":"false"
},
"PRODUCT_NAME": {
  "type": "keyword","index":"false"
},
"PREMIUM_AMOUNT_INR_WITHOUT_TAX": {
  "type": "double","index":"false"
},
"CGST_AMOUNT": {
  "type": "double","index":"false"
},
"IGST_AMOUNT": {
  "type": "double","index":"false"
},
"SGST_AMOUNT": {
  "type": "double","index":"false"
},
"PREMIUM_WITH_TAX": {
  "type": "double","index":"false"
},
"SUM_INSURED": {
  "type": "double","index":"false"
},
"PRODUCER_CD": {
  "type": "keyword","index":"false"
},
"PRODUCER_NAME": {
  "type": "keyword","index":"false"
},
"BRANCH": {
  "type": "keyword","index":"false"
},
"RECORD_TYPE_DESC": {
  "type": "keyword","index":"false"
},
"LOB": {
  "type": "keyword","index":"false"
},
"VEHICLE_REGISTRATION_NUMBER": {
  "type": "keyword","index":"false"
},
"IPDS_POLICY_NUMBER": {
  "type": "keyword"
},
"POLICY_STATUS": {
  "type": "keyword","index":"false"
},
"PROPOSAL_DESCRIPTION": {
  "type": "keyword","index":"false"
},
"RENL_CERT_NO": {
  "type": "keyword","index":"false"
},
"EFF_DT_SEQ_NO": {
  "type": "keyword","index":"false"
},
"TXT_TIMES_RENEWED": {
  "type": "keyword","index":"false"
},
"PREVIOUS_POL_START_DATE": {
  "type": "keyword","index":"false"
},
"PREVIOUS_POL_EXPIRY_DATE": {
  "type": "keyword","index":"false"
},
"DRIVING_LICENSE_NUMBER": {
  "type": "keyword","index":"false"
},
"PORTABILITY_FLAG": {
  "type": "keyword","index":"false"
},
"DC_UNIFIED_ID": {
  "type": "keyword","index":"false"
},
"MOBILE_NUMBER": {
  "type": "keyword"
},
"EMAIL_ID": {
  "type": "keyword"
},
"CUSTOMER_NAME": {
  "type": "keyword","index":"false"
},
"LOAD_DATE": {
  "type": "keyword"
},
"primary_key": {
"type": "keyword"
},
}
}}



URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index+'/_mapping/'+target_doc_type
headers = {'Content-Type': 'application/json'}
r = requests.put(URL, data=json.dumps(mapping_output), headers=headers)
print(r)

In [ ]:
try:
     policy_retail_base_main1.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
        .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
        .option("es.write.operation","upsert").option("es.mapping.id",'primary_key').mode("append").save() 
except Exception as e :
    x = e
else: 
    x = 200 # success
print(x)

In [ ]:
table_name='spav2_policy'
source='spav2_policy'
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod

In [ ]:
# Generic function to save the progress from last run into Progress table
def update_progress(table_name,source,time_filter,max_commit_timestamp,records,start_time,starttime,status):
    output_index = "progress"
    schema = "customermart"


    import sys
    import time
    try:
        


        df_progress= sqlContext.createDataFrame([(table_name,source,
                                       time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S"),
                                        max_commit_timestamp.strftime("%Y-%m-%d %H:%M:%S"),records)]
                                     ,['table_name', 'source','from_datetime','to_datetime','records'])\
        .withColumn('start_time',lit(start_time)).withColumn('end_time',current_timestamp())\
                    .withColumn('executed_in_mins',lit((time.time() - starttime)/60))\
                                .withColumn('status',lit(status)).withColumn('remarks',lit(None))
    

        
        df_progress.write.format("greenplum")\
            .option("dbtable",output_index).option('dbschema',schema)\
            .option("server.port","1150-1170").option("url",prod_url)\
            .option("user", user_prod).option("password",pwd_prod).mode('append').save()
        
       
    except Exception as e:
        x = e
        print(x)
    else:
        x = 'success'
        print("Updated Progress for {table_name} from {source} until {to_datetime}. {records} records processed in this run".format(table_name=table_name, source=source,to_datetime=max_commit_timestamp.strftime("%Y-%m-%d %H:%M:%S"),records=records))
        print(x)

In [ ]:
if  x == 200:
    update_progress(table_name,source,time_filter,max_commit_timestamp,records,start_time,starttime,'Success')
    print('SUCCESS')
else:
    update_progress(table_name,source,time_filter,max_commit_timestamp,'0',start_time,starttime,'Failed')
    print('FAILED')

In [ ]:
policy_retail_base_main1.unpersist()

In [ ]:
import io
import pandas as pd
import xlsxwriter
import sys
import smtplib
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
import pandas as pd
from sqlalchemy import create_engine
from smtplib import SMTP

if x!=200:
    status='FAILED'
else:
    status="SUCCESS"

subject=f"Automated Mail-SPA PART-2 Job Status : {status}"

if x==200:
    recipients = ['nishant5.addenki@tataaig.com','devnandan.TataIndustries@tataaig.com','mayur.TataIndustries@tataaig.com','Ajeet3.Jaiswal@tataaig.com'] 
    emaillist = [elem.strip().split(',') for elem in recipients]
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = 'customercoe@tataaig.com'


    html = """\
    <html>
      <head></head>
      <body>
        SPA PART-2 job status is: Successful
      </body>
        <p>This is automated mail, please don't reply. Thanks</p>
    </html>
    """

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    server = smtplib.SMTP("tataaig-com.mail.protection.outlook.com",25)
    server.sendmail(msg['From'], emaillist , msg.as_string())
else:
      
    recipients = ['nishant5.addenki@tataaig.com','devnandan.TataIndustries@tataaig.com','mayur.TataIndustries@tataaig.com','Ajeet3.Jaiswal@tataaig.com'] 
    emaillist = [elem.strip().split(',') for elem in recipients]
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = 'customercoe@tataaig.com'


    html = """\
    <html>
      <head></head>
      <body>
        CPA job status is: Failed
      </body>
        <p>This is automated mail, please don't reply. Thanks</p>
    </html>
    """

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    server = smtplib.SMTP("tataaig-com.mail.protection.outlook.com",25)
    server.sendmail(msg['From'], emaillist , msg.as_string())

In [ ]:
spark.stop()